# Carregando Ambiente

In [ ]:
// Importando as bibliotecas necessárias
const faker = require('faker');
const mongoose = require('mongoose');
mongoose.set('useCreateIndex', true);
const _ = require('lodash');
const countryData = require('country-data');

// Conectando a base de dados usando mongoose
mongoose.connect('mongodb://localhost/effective_store', { useUnifiedTopology: true, useNewUrlParser: true }).then(() => console.log('Connected'));
console.log('Connecting...')

# Subindo Dados e Schemas na Base

In [ ]:
// Definindo os países disponíveis
const availableCountries = _.filter(countryData.countries.all, countrie => countrie.status === 'assigned');

// Definindo o status de pagamento disponível
const availableStatus = ['Payed', 'Waiting Payment', 'Canceled']

// Definindo schema de produto
const ProductSchema = new mongoose.Schema({
  name: String,
  description: String,
  price: Number
});

// Definindo o schema de Invoice
const InvoiceSchema = new mongoose.Schema({
  invoiceCode: {
    type: String,
    unique: true
  },
  dueDate: Date,
  status: {
    type: String,
    enum: availableStatus
  },
  productList: [ProductSchema],
  totalAmount: Number,
  invoiceDoc: {
    sellerCode: {
      type: String,
      unique: true
    },
    docCode: { 
      type: String,
      unique: true
    }
  }
})

// Definindo o model de Invoice
const Invoice = mongoose.model('Invoice', InvoiceSchema);

// Definindo o schema de Sellers
const SellerSchema = new mongoose.Schema({
  code: {
    type: String,
    unique: true
  },
  name: String,
  country: {
    code: { type: String, enum: availableCountries.map(country => country.alpha2) },
    name: { type: String, enum: availableCountries.map(country => country.name) },
    emojiFlag: { type: String, enum: availableCountries.map(country => country.emoji) }
  },
  invoices: [InvoiceSchema],
  createdAt: Date,
  productCatalog: [ProductSchema]
});

// Definindo o model de Seller
const Seller = mongoose.model('Seller', SellerSchema);


console.log('Done!');

# Gerando dados de sellers

In [ ]:
const SELLERS_LENGTH = 500;
const PRODUCTS_LENGTH = 100;

const availableProducts = _.range(PRODUCTS_LENGTH).map(() => ({
  name: faker.commerce.product(),
  price: parseFloat(faker.commerce.price()),
  description: faker.commerce.productDescription(),
}));

const sellers = _.range(SELLERS_LENGTH).map((sellerIndex) => {
  const productCatalogSize = _.random(1, PRODUCTS_LENGTH)
  const country = _.sample(availableCountries)
  const productCatalog = []
  _.range(productCatalogSize).forEach(() => {
    const productSample = _.sample(availableProducts)
    if (!productCatalog.includes(productSample)) {
      productCatalog.push(productSample)
    }
  });
  
  return new Seller({
    code: _.uniqueId(`seller_cod_${sellerIndex}_`),
    name: faker.company.companyName(),
    country: { code: country.alpha2, name: country.name, emojiFlag: country.emoji },
    createdAt: faker.date.past(10),
    productCatalog
  });
});
console.log('Done!');

# Gerando dados de invoices

In [ ]:
const MAX_SALES_BY_A_SELLER = 5

const salesNumbers = _.range(MAX_SALES_BY_A_SELLER)
const setSellerInvoice = seller => {
  const invoices = []
  _.range(_.sample(salesNumbers)).forEach((index) => {
    const productCatalogRange = _.range(seller.productCatalog.length)
    const invoiceProductsLength = _.sample(productCatalogRange)
    const productList = _.range(invoiceProductsLength).map(() => _.sample(seller.productCatalog))
    if (!_.isEmpty(productList)) {
      invoices.push(new Invoice({
        invoiceCode: _.uniqueId(`invoice_${seller.code}`),
        dueDate: faker.date.past(10),
        status: _.sample(availableStatus),
        productList,
        totalAmount: _.reduce(_.map(productList, product => product.price), (price, memo) => price + memo , 0),
        invoiceDoc: {
          sellerCode: seller.code,
          docCode: _.uniqueId(`doc_${seller.code}`)
        }
      }))
    }
  })

  seller.invoices = invoices
}

sellers.forEach(setSellerInvoice)
console.log('Done!');

# Populando base de dados
Limpamos a collection de sellers e após isto inserimos os dados gerados
## Atenção aguardar o log de Inserted

In [ ]:
console.log('Cleaning sellers collection')
Seller.deleteMany({}, () => {
  console.log('cleared')
  console.log('Inserting generated data')
  Seller.insertMany(sellers, () => console.log('Inserted') )
})

console.log('Processing...');

# Query 1 - Lojistas (Sellers) com maior número de produtos disponíveis na plataforma

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      productCatalogSize: { $cond: { if: { $isArray: "$productCatalog" }, then: { $size: "$productCatalog" }, else: "NA"} }
    }
  },
  {
    $sort : { productCatalogSize : -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...');

# Query 2 - Produtos mais comuns entre os lojistas associados

In [ ]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...');

# Query 3 - Países com mais lojistas

In [ ]:
Seller.aggregate([
  {
    $group: {
      _id: '$country.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...');

# Query 4 - Produtos com mais lojistas por região

In [ ]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      country: { $first : "$country.name" },
      counts: { $sum: 1 },
    }
  },
  {
    $sort: { country: -1 }
  }
], (errors, result) => console.table(result));
console.log('Processing...');

# Query 5 - Lista dos lojistas (Sellers) ordenada pelos mais novos

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result));
console.log('Processing...');

# Query 6 - Lista dos lojistas (Sellers) ordenada pelos mais antiga

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: 1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result));
console.log('Processing...');

# Query 7 - Lista das faturas que estão em aberto para os lojistas(Sellers) com menos de 1 ano de mercado

In [ ]:
Seller.aggregate([
  {
    $unwind: '$invoices'
  },
  {
    $limit: 1
  }
], (errors, result) => console.log(result))
console.log('Processing...');

# Query 8 - Lista das faturas acima de 1000,00